In [2]:
import polars as pl
import numpy as np
from datetime import datetime, timedelta

In [3]:
np.random.seed(42)
def generate_coffee_data():
    n_records = 2000  # 2000 transactions over 6 months
    #Menue:
    menu_items = ['Espresso', 'Cappuccino', 'Latte', 'Americano', 'Mocha', 'Cold Brew']
    prices = [2.50, 4.00, 4.50, 3.00, 5.00, 3.50]
    # Getting dates over 6 months
    start_date = datetime(2023, 6, 1)
    dates = [start_date + timedelta(days=np.random.randint(0, 180))
            for _ in range(n_records)]
    data = {
        'date': dates,
        'drink': np.random.choice(menu_items, n_records),
        'price': [prices[menu_items.index(drink)] for drink in
                  np.random.choice(menu_items, n_records)],
        'quantity': np.random.choice([1, 1, 1, 2, 2, 3], n_records),  # Most people buy 1
        'customer_type': np.random.choice(['Regular', 'New', 'Tourist'],
                                        n_records, p=[0.5, 0.3, 0.2]),
        'payment_method': np.random.choice(['Card', 'Cash', 'Mobile'],
                                         n_records, p=[0.6, 0.2, 0.2]),
        'rating': np.random.choice([2,3, 4, 5], n_records, p=[0.1, 0.4, 0.4, 0.1])
    }
    return data
# Coffee shop DataFrame
coffee_data = generate_coffee_data()
df = pl.DataFrame(coffee_data)

In [4]:
print("First 5 transactions:")
print(df.head())

print("\nTypes of data?")
print(df.schema)

print("\nHow big is the dataset?")
print(f"We have {df.height} transactions and {df.width} columns")

First 5 transactions:
shape: (5, 7)
┌─────────────────────┬────────────┬───────┬──────────┬───────────────┬────────────────┬────────┐
│ date                ┆ drink      ┆ price ┆ quantity ┆ customer_type ┆ payment_method ┆ rating │
│ ---                 ┆ ---        ┆ ---   ┆ ---      ┆ ---           ┆ ---            ┆ ---    │
│ datetime[μs]        ┆ str        ┆ f64   ┆ i32      ┆ str           ┆ str            ┆ i32    │
╞═════════════════════╪════════════╪═══════╪══════════╪═══════════════╪════════════════╪════════╡
│ 2023-09-11 00:00:00 ┆ Cold Brew  ┆ 5.0   ┆ 1        ┆ New           ┆ Cash           ┆ 4      │
│ 2023-11-27 00:00:00 ┆ Cappuccino ┆ 4.5   ┆ 1        ┆ New           ┆ Card           ┆ 4      │
│ 2023-09-01 00:00:00 ┆ Espresso   ┆ 4.5   ┆ 1        ┆ Regular       ┆ Card           ┆ 3      │
│ 2023-06-15 00:00:00 ┆ Cappuccino ┆ 5.0   ┆ 1        ┆ New           ┆ Card           ┆ 4      │
│ 2023-09-15 00:00:00 ┆ Mocha      ┆ 5.0   ┆ 2        ┆ Regular       ┆ Card      